### Ngô Hồng Thông 22649011

Từ bộ dữ liệu lưu lượng hành khách sử dụng dịch vụ hàng không qua các năm tại [international airline passengers](https://raw.githubusercontent.com/phamdinhkhanh/LSTM/master/international-airline-passengers.csv) bạn hãy:

1. Phân chia tập huấn luyện/kiểm tra sao cho tập kiểm tra bao gồm 12 tháng cuối cùng và tập huấn luyện gồm các tháng trước đó.
2. Xây dựng phương trình dự báo lưu lượng hành khách theo phương trình hồi qui tuyến tính đơn biến trên tập huấn luyện và đánh giá MSE trên tập kiểm tra.
3. Tạo thêm các biến $x^2, x^3$ và xây dựng phương trình hồi qui tuyến tính đa biến.
4. Huấn luyên mô hình với Ridge Regression và Lasso Regression. Fine tunning hệ số $\alpha$ của thành phần điều chuẩn.

In [45]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [46]:
url = 'https://raw.githubusercontent.com/phamdinhkhanh/LSTM/master/international-airline-passengers.csv'
df = pd.read_csv(url)
df.head()

,Month,International airline passengers: monthly totals in thousands. Jan 49 ? Dec 60
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [47]:
df = df.dropna()

In [48]:
df.describe()

,International airline passengers: monthly totals in thousands. Jan 49 ? Dec 60
count,144.000000
mean,280.298611
std,119.966317
min,104.000000
25%,180.000000
50%,265.500000
75%,360.500000
max,622.000000


In [49]:
df.columns = ['date', 'visitor']


In [50]:
#1.
test_df = df[-12:]
train_df = df[:-12]

test_df, train_df

(        date  visitor
 132  1960-01      417
 133  1960-02      391
 134  1960-03      419
 135  1960-04      461
 136  1960-05      472
 137  1960-06      535
 138  1960-07      622
 139  1960-08      606
 140  1960-09      508
 141  1960-10      461
 142  1960-11      390
 143  1960-12      432,
         date  visitor
 0    1949-01      112
 1    1949-02      118
 2    1949-03      132
 3    1949-04      129
 4    1949-05      121
 ..       ...      ...
 127  1959-08      559
 128  1959-09      463
 129  1959-10      407
 130  1959-11      362
 131  1959-12      405
 
 [132 rows x 2 columns])

In [51]:
#2.

# Hàm tính MSE
def MSE(y_test, y_pred):
    return np.mean((np.array(y_test) - np.array(y_pred)) ** 2)

# Nội suy giá trị bị thiếu trong cột 'visitor'
df['visitor'] = df['visitor'].interpolate()

# Thêm cột 'date_index' để làm biến độc lập
df['date_index'] = np.arange(1, len(df) + 1)

# Phân chia tập huấn luyện và tập kiểm tra
train_df = df.iloc[:-12]
test_df = df.iloc[-12:]

# Lấy biến đầu vào (date_index) và biến dự báo (visitor)
X_train = train_df[['date_index']]
y_train = train_df['visitor']

X_test = test_df[['date_index']]
y_test = test_df['visitor']

# Xây dựng mô hình hồi quy tuyến tính
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = lin_reg.predict(X_test)

# Tính toán MSE trên tập kiểm tra
mse = MSE(y_test, y_pred)

# Phương trình hồi quy
coef = lin_reg.coef_[0]
intercept = lin_reg.intercept_

print(f'MSE = {mse:.2f}')
print(f'Phương trình hồi quy: y = {coef:.2f}x + {intercept:.2f}')


MSE = 6213.02
Phương trình hồi quy: y = 2.56x + 92.01


In [52]:
#3. 
df['date_index_2'] = df['date_index'] ** 2
df['date_index_3'] = df['date_index'] ** 3

In [53]:
# Lấy biến đầu vào (date_index, date_index^2, date_index^3) và biến dự báo (visitor)

#Phân chia train/test
test_df = df[-12:]
train_df = df[:-12]

X_train = train_df[['date_index', 'date_index_2', 'date_index_3']]
y_train = train_df['visitor']

X_test = test_df[['date_index', 'date_index_2', 'date_index_3']]
y_test = test_df['visitor']

# Xây dựng mô hình hồi quy tuyến tính đa biến
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = lin_reg.predict(X_test)

# Tính toán MSE trên tập kiểm tra
mse = MSE(y_test, y_pred)

# Phương trình hồi quy đa biến
coef = lin_reg.coef_
intercept = lin_reg.intercept_

print('MSE = ', mse)
print(f'Phương trình hồi quy: y = {coef[0]:.2f}x + {coef[1]:.2f}x^2 + ({coef[2]:.2f})x^3 + {intercept:.2f}')


MSE =  5663.456143470688
Phương trình hồi quy: y = 0.94x + 0.02x^2 + (-0.00)x^3 + 120.00


In [54]:
# 4.

# Ridge Regression với fine-tune alpha
ridge = Ridge()
alpha_values = {'alpha': [0.01, 0.01, 0.1, 1, 10, 100]}
ridge_grid = GridSearchCV(ridge, alpha_values, cv=5, scoring='neg_mean_squared_error')
ridge_grid.fit(X_train, y_train)

# Dự đoán và tính MSE cho Ridge
y_pred_ridge = ridge_grid.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
best_alpha_ridge = ridge_grid.best_params_['alpha']

# In kết quả Ridge
print(f'Ridge Regression - MSE: {mse_ridge:.2f}, Best alpha: {best_alpha_ridge}')

Ridge Regression - MSE: 5683.70, Best alpha: 100


In [55]:
# Lasso Regression với fine-tune alpha
lasso = Lasso()
lasso_grid = GridSearchCV(lasso, alpha_values, cv=5, scoring='neg_mean_squared_error')
lasso_grid.fit(X_train, y_train)

# Dự đoán và tính MSE cho Lasso
y_pred_lasso = lasso_grid.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
best_alpha_lasso = lasso_grid.best_params_['alpha']

# In kết quả Lasso
print(f'Lasso Regression - MSE: {mse_lasso:.2f}, Best alpha: {best_alpha_lasso}')

Lasso Regression - MSE: 6152.09, Best alpha: 100


c:\Users\thong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.057e+05, tolerance: 9.360e+01
  model = cd_fast.enet_coordinate_descent(
c:\Users\thong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.032e+05, tolerance: 1.316e+02
  model = cd_fast.enet_coordinate_descent(
c:\Users\thong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca